analysis job vision data for jobs in iran 

__importing libraries__

In [63]:
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly


In [47]:

df = pd.read_csv("C:\\Users\\Sepehr\\ML_file\\JobVision\\JobVision_Jobposts_Dataset.csv")

df.head()

,RowNumber,Jobpost_RawTitle,Jobpost_PreferredGender,Jobpost_ProvinceFa,Jobpost_ProvinceEn,Jobpost_WorkTypeFa,Jobpost_WorkTypeEn,Jobpost_SalaryCanBeShown,Jobpost_MinSalary,Jobpost_MaxSalary,...,Company_CityFa,Company_CityEn,Comany_CompanyOwnershipTypesFa,Company_CompanyOwnershipTypesEn,Company_SizeFa,Company_SizeEn,Company_ActivityTypeFa,Company_ActivityTypeEn,Company_IndustryFa,Company_IndustryEn
0,1,مهندس مکانیک,تفاوتی ندارد,تهران,Tehran,پاره وقت,Part Time,False,NaN,NaN,...,تهران,Tehran,خصوصی,Privately held,11 تا 50 نفر,11 - 50 employees,شرکت ایرانی دارای مشتریان داخلی,Iranian company dealing only with Iranian enti...,"ساختمان / مصالح و تجهیزات ساختمانی,تولیدی / صنعتی","Construction / Building Materials & Equipment,..."
1,2,طراح و گرافیست,تفاوتی ندارد,تهران,Tehran,تمام وقت,Full Time,False,NaN,NaN,...,تهران,Tehran,خصوصی,Privately held,11 تا 50 نفر,11 - 50 employees,شرکت ایرانی دارای مشتریان داخلی و خارجی,Iranian company dealing with Iranian and forei...,طراحی / گرافیک / هنر,Design / Arts
2,3,برنامه نویس full-stack,تفاوتی ندارد,خراسان رضوی,Khorasan Razavi,تمام وقت,Full Time,False,NaN,NaN,...,مشهد,Mashhad,خصوصی,Privately held,زیر 10 نفر,Less than 10 employees,شرکت ایرانی دارای مشتریان داخلی و خارجی,Iranian company dealing with Iranian and forei...,"اینترنت / تجارت الکترونیک / خدمات آنلاین,شرکت ...",Internet Provider / E-commerce / Online Servic...
3,4,بازاریاب - خانم,فقط خانم,البرز,Alborz,تمام وقت,Full Time,False,NaN,NaN,...,کرج,Karaj,خصوصی,Privately held,زیر 10 نفر,Less than 10 employees,شرکت ایرانی دارای مشتریان داخلی,Iranian company dealing only with Iranian enti...,"اینترنت / تجارت الکترونیک / خدمات آنلاین,خرده ...",Internet Provider / E-commerce / Online Servic...
4,5,برنامه نویس React Native,تفاوتی ندارد,اصفهان,Isfahan,پاره وقت,Part Time,False,NaN,NaN,...,اصفهان,Isfahan,خصوصی,Privately held,501 تا 1000 نفر,501 - 1000 employees,شرکت ایرانی دارای مشتریان داخلی و خارجی,Iranian company dealing with Iranian and forei...,فناوری اطلاعات / نرم افزار و سخت افزار,IT / Software / Hardware


In [48]:
df.describe()

,RowNumber,Jobpost_MinSalary,Jobpost_MaxSalary,Jobpost_RequiredExperienceYears,Jobpost_RequiredMinAge,Jobpost_RequiredMaxAge
count,364838.000000,120808.000000,120808.000000,364838.000000,268475.000000,277556.000000
mean,182419.500000,11.779344,15.053043,2.174957,23.673798,38.102437
std,105319.803092,6.166681,8.310282,1.890549,3.805943,5.414457
min,1.000000,2.000000,3.000000,0.000000,18.000000,18.000000
25%,91210.250000,8.000000,10.000000,1.000000,20.000000,35.000000
50%,182419.500000,10.000000,12.000000,2.000000,24.000000,37.000000
75%,273628.750000,15.000000,20.000000,3.000000,25.000000,40.000000
max,364838.000000,70.000000,100.000000,30.000000,50.000000,60.000000


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364838 entries, 0 to 364837
Data columns (total 43 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   RowNumber                                        364838 non-null  int64  
 1   Jobpost_RawTitle                                 364838 non-null  object 
 2   Jobpost_PreferredGender                          364838 non-null  object 
 3   Jobpost_ProvinceFa                               364838 non-null  object 
 4   Jobpost_ProvinceEn                               364838 non-null  object 
 5   Jobpost_WorkTypeFa                               364838 non-null  object 
 6   Jobpost_WorkTypeEn                               364838 non-null  object 
 7   Jobpost_SalaryCanBeShown                         364838 non-null  bool   
 8   Jobpost_MinSalary                                120808 non-null  float64
 9   Jobpost_MaxSala

In [50]:
df.columns

Index(['RowNumber', 'Jobpost_RawTitle', 'Jobpost_PreferredGender',
       'Jobpost_ProvinceFa', 'Jobpost_ProvinceEn', 'Jobpost_WorkTypeFa',
       'Jobpost_WorkTypeEn', 'Jobpost_SalaryCanBeShown', 'Jobpost_MinSalary',
       'Jobpost_MaxSalary', 'Jobpost_MainJobCategory',
       'Jobpost_SecondaryJobCategories', 'Jobpost_IndustryFa',
       'Jobpost_IndustryEn', 'Jobpost_RequiredExperienceYears',
       'Jobpost_RequiredRelatedExperienceInThisIndustry',
       'Jobpost_CompanyWorkTimesDescription', 'Jobpost_LanguageSkills',
       'Jobpost_SoftwareSkills', 'Jobpost_AcademicFields', 'Jobpost_BenefitFa',
       'Jobpost_BenefitEn', 'Jobpost_HasDisabilitySupport',
       'Jobpost_RequiredMinAge', 'Jobpost_RequiredMaxAge',
       'Jobpost_RequiredMilitaryServiceCard', 'Jobpost_IsRemote',
       'Jobpost_IsInternship', 'Jobpost_PriorityWithLocalCandidate',
       'Jobpost_ActivationTime_YEAR_MONTH', 'Company_AgeFromEstablishmentYear',
       'Company_ProvinceFa', 'Company_ProvinceEn', 'Comp

In [51]:
new_columns = ['Jobpost_RawTitle',
    'Jobpost_MainJobCategory',
    'Jobpost_IndustryFa',
    'Jobpost_WorkTypeFa',
    'Jobpost_IsRemote',
    'Jobpost_IsInternship',
    'Jobpost_PreferredGender',
    'Jobpost_RequiredExperienceYears',
    'Jobpost_RequiredRelatedExperienceInThisIndustry',
    'Jobpost_RequiredMaxAge',
    'Jobpost_RequiredMilitaryServiceCard',
    'Jobpost_LanguageSkills',
    'Jobpost_SoftwareSkills',
    'Jobpost_AcademicFields',
    'Jobpost_SalaryCanBeShown',
    'Jobpost_MinSalary',
    'Jobpost_MaxSalary',
    'Jobpost_ProvinceFa',
    'Company_ProvinceFa',
    'Company_CityFa',
    'Comany_CompanyOwnershipTypesFa',  
    'Company_SizeFa',
    'Company_ActivityTypeFa',
    'Company_IndustryFa',
    'Company_AgeFromEstablishmentYear',
    'Jobpost_ActivationTime_YEAR_MONTH',
    'Jobpost_PriorityWithLocalCandidate'
]



In [52]:
df_new = df[new_columns].copy()

In [53]:
df_new.columns

Index(['Jobpost_RawTitle', 'Jobpost_MainJobCategory', 'Jobpost_IndustryFa',
       'Jobpost_WorkTypeFa', 'Jobpost_IsRemote', 'Jobpost_IsInternship',
       'Jobpost_PreferredGender', 'Jobpost_RequiredExperienceYears',
       'Jobpost_RequiredRelatedExperienceInThisIndustry',
       'Jobpost_RequiredMaxAge', 'Jobpost_RequiredMilitaryServiceCard',
       'Jobpost_LanguageSkills', 'Jobpost_SoftwareSkills',
       'Jobpost_AcademicFields', 'Jobpost_SalaryCanBeShown',
       'Jobpost_MinSalary', 'Jobpost_MaxSalary', 'Jobpost_ProvinceFa',
       'Company_ProvinceFa', 'Company_CityFa',
       'Comany_CompanyOwnershipTypesFa', 'Company_SizeFa',
       'Company_ActivityTypeFa', 'Company_IndustryFa',
       'Company_AgeFromEstablishmentYear', 'Jobpost_ActivationTime_YEAR_MONTH',
       'Jobpost_PriorityWithLocalCandidate'],
      dtype='object')

In [54]:
del df


In [55]:
pd.reset_option('display.max_columns')
df_new.head(24)

,Jobpost_RawTitle,Jobpost_MainJobCategory,Jobpost_IndustryFa,Jobpost_WorkTypeFa,Jobpost_IsRemote,Jobpost_IsInternship,Jobpost_PreferredGender,Jobpost_RequiredExperienceYears,Jobpost_RequiredRelatedExperienceInThisIndustry,Jobpost_RequiredMaxAge,...,Jobpost_ProvinceFa,Company_ProvinceFa,Company_CityFa,Comany_CompanyOwnershipTypesFa,Company_SizeFa,Company_ActivityTypeFa,Company_IndustryFa,Company_AgeFromEstablishmentYear,Jobpost_ActivationTime_YEAR_MONTH,Jobpost_PriorityWithLocalCandidate
0,مهندس مکانیک,مهندسی مکانیک / مهندسی هوا و فضا,تولیدی / صنعتی,پاره وقت,False,False,تفاوتی ندارد,1,False,35.0,...,تهران,تهران,تهران,خصوصی,11 تا 50 نفر,شرکت ایرانی دارای مشتریان داخلی,"ساختمان / مصالح و تجهیزات ساختمانی,تولیدی / صنعتی",more than 20 years,2022-04,False
1,طراح و گرافیست,طراحی گرافیک / طراحی انیمیشن و موشن گرافیک,طراحی / گرافیک / هنر,تمام وقت,False,False,تفاوتی ندارد,3,False,33.0,...,تهران,تهران,تهران,خصوصی,11 تا 50 نفر,شرکت ایرانی دارای مشتریان داخلی و خارجی,طراحی / گرافیک / هنر,10 - 14 years,2023-03,True
2,برنامه نویس full-stack,توسعه نرم افزار و برنامه نویسی,اینترنت / تجارت الکترونیک / خدمات آنلاین,تمام وقت,False,False,تفاوتی ندارد,3,True,35.0,...,خراسان رضوی,خراسان رضوی,مشهد,خصوصی,زیر 10 نفر,شرکت ایرانی دارای مشتریان داخلی و خارجی,"اینترنت / تجارت الکترونیک / خدمات آنلاین,شرکت ...",5 - 9 years,2022-04,False
3,بازاریاب - خانم,فروش و بازاریابی - فروشنده / بازاریاب و ویزیتو...,اینترنت / تجارت الکترونیک / خدمات آنلاین,تمام وقت,False,False,فقط خانم,0,False,40.0,...,البرز,البرز,کرج,خصوصی,زیر 10 نفر,شرکت ایرانی دارای مشتریان داخلی,"اینترنت / تجارت الکترونیک / خدمات آنلاین,خرده ...",10 - 14 years,2023-12,True
4,برنامه نویس React Native,توسعه نرم افزار و برنامه نویسی,فناوری اطلاعات / نرم افزار و سخت افزار,پاره وقت,False,False,تفاوتی ندارد,1,True,30.0,...,اصفهان,اصفهان,اصفهان,خصوصی,501 تا 1000 نفر,شرکت ایرانی دارای مشتریان داخلی و خارجی,فناوری اطلاعات / نرم افزار و سخت افزار,more than 20 years,2022-07,True
5,طراح وب وردپرس - خانم,دیجیتال مارکتینگ و سئو,فناوری اطلاعات / نرم افزار و سخت افزار,تمام وقت,False,False,فقط خانم,3,True,35.0,...,تهران,تهران,تهران,خصوصی,زیر 10 نفر,شرکت ایرانی دارای مشتریان داخلی,فناوری اطلاعات / نرم افزار و سخت افزار,10 - 14 years,2023-11,True
6,کارشناس ارشد امور بانکی و ارزی و سامانه نیما,مالی و حسابداری,تجارت / بازرگانی,تمام وقت,False,False,تفاوتی ندارد,3,True,50.0,...,تهران,تهران,تهران,خصوصی,زیر 10 نفر,شرکت ایرانی دارای مشتریان داخلی,تجارت / بازرگانی,5 - 9 years,2022-04,False
7,کارشناس ارشد خرید خارجی (فلزات گرانبها),بازرگانی / تجارت,تجارت / بازرگانی,تمام وقت,False,False,تفاوتی ندارد,3,True,50.0,...,تهران,تهران,تهران,خصوصی,زیر 10 نفر,شرکت ایرانی دارای مشتریان داخلی,تجارت / بازرگانی,5 - 9 years,2022-04,False
8,کارشناس ارشد خرید خارجی (قطعات نفت، گاز و پترو...,خرید / تدارکات,تجارت / بازرگانی,تمام وقت,False,False,تفاوتی ندارد,3,True,50.0,...,تهران,تهران,تهران,خصوصی,زیر 10 نفر,شرکت ایرانی دارای مشتریان داخلی,تجارت / بازرگانی,5 - 9 years,2022-04,False
9,طراح سایت وردپرس - خانم,توسعه نرم افزار و برنامه نویسی,فناوری اطلاعات / نرم افزار و سخت افزار,تمام وقت,False,False,فقط خانم,2,True,35.0,...,تهران,تهران,تهران,خصوصی,زیر 10 نفر,شرکت ایرانی دارای مشتریان داخلی,فناوری اطلاعات / نرم افزار و سخت افزار,10 - 14 years,2023-11,True


In [56]:
df_new.isnull().sum()

Jobpost_RawTitle                                        0
Jobpost_MainJobCategory                                 0
Jobpost_IndustryFa                                      0
Jobpost_WorkTypeFa                                      0
Jobpost_IsRemote                                        0
Jobpost_IsInternship                                    0
Jobpost_PreferredGender                                 0
Jobpost_RequiredExperienceYears                         0
Jobpost_RequiredRelatedExperienceInThisIndustry         0
Jobpost_RequiredMaxAge                              87282
Jobpost_RequiredMilitaryServiceCard                     0
Jobpost_LanguageSkills                             235240
Jobpost_SoftwareSkills                             168264
Jobpost_AcademicFields                             193088
Jobpost_SalaryCanBeShown                                0
Jobpost_MinSalary                                  244030
Jobpost_MaxSalary                                  244030
Jobpost_Provin

In [58]:
df_new[["Jobpost_LanguageSkills","Jobpost_SoftwareSkills","Jobpost_AcademicFields"]].head(20)
  

,Jobpost_LanguageSkills,Jobpost_SoftwareSkills,Jobpost_AcademicFields
0,NaN,NaN,"[{""FieldType"":""مهندسی مکانیک"",""DegreeLevel"":""ک..."
1,NaN,"[{""TitleFa"":""ایلاستریتور,ایلوستریتور"",""TitleEn...","[{""FieldType"":""هنر"",""DegreeLevel"":""کارشناسی""}]"
2,NaN,"[{""TitleFa"":""اچ تی ام ال,سی اس اس"",""TitleEn"":""...","[{""FieldType"":""کامپیوتر - فناوری اطلاعات"",""Deg..."
3,NaN,NaN,NaN
4,NaN,"[{""TitleFa"":""ری اکت"",""TitleEn"":""React"",""Level""...",NaN
5,NaN,"[{""TitleFa"":""اکسس"",""TitleEn"":""Microsoft Access...",NaN
6,NaN,NaN,NaN
7,"[{""Languages_TitleFa"":""انگلیسی"",""Languages_Tit...",NaN,NaN
8,NaN,NaN,NaN
9,NaN,"[{""TitleFa"":""وردپرس"",""TitleEn"":""Wordpress"",""Le...",NaN


In [62]:
df_new['Company_ActivityTypeFa'].value_counts()


Company_ActivityTypeFa
شرکت ایرانی دارای مشتریان داخلی و خارجی    146047
شرکت ایرانی دارای مشتریان داخلی            125232
شعبه ای از یک شرکت خارجی / سفارتخانه         6047
Name: count, dtype: int64

Start Cleaning data


In [70]:
num_cols = ['Jobpost_MinSalary', 'Jobpost_MaxSalary']

text_cols_fill_not_mentioned = [
    'Jobpost_LanguageSkills',
    'Jobpost_SoftwareSkills',
    'Jobpost_AcademicFields',
    'Company_CityFa'
]

mode_fill_cols = [
    'Jobpost_RequiredMaxAge',
    'Comany_CompanyOwnershipTypesFa'  
]
activity_col = 'Company_ActivityTypeFa'


In [65]:
num_imputer = SimpleImputer(strategy='mean')
df_new[num_cols] = num_imputer.fit_transform(df_new[num_cols])

In [66]:

text_imputer = SimpleImputer(strategy='constant', fill_value='not mentioned')
df_new[text_cols_fill_not_mentioned] = text_imputer.fit_transform(df_new[text_cols_fill_not_mentioned])

In [67]:
for col in mode_fill_cols:
    if col in df_new.columns:
        mode_value = df_new[col].mode()
        if not mode_value.empty:
            fill_value = mode_value[0]
        else:
            fill_value = 'نامشخص'
        df_new[col] = df_new[col].fillna(fill_value)

In [71]:
if activity_col in df_new.columns:
    freq = df_new[activity_col].value_counts()
    if len(freq) > 2:
        freq_middle = freq.iloc[1:-1]
        fill_value = freq_middle.idxmax()
    else:
        fill_value = df_new[activity_col].mode()[0]
    df_new[activity_col] = df_new[activity_col].fillna(fill_value)

In [72]:
df_new.columns

Index(['Jobpost_RawTitle', 'Jobpost_MainJobCategory', 'Jobpost_IndustryFa',
       'Jobpost_WorkTypeFa', 'Jobpost_IsRemote', 'Jobpost_IsInternship',
       'Jobpost_PreferredGender', 'Jobpost_RequiredExperienceYears',
       'Jobpost_RequiredRelatedExperienceInThisIndustry',
       'Jobpost_RequiredMaxAge', 'Jobpost_RequiredMilitaryServiceCard',
       'Jobpost_LanguageSkills', 'Jobpost_SoftwareSkills',
       'Jobpost_AcademicFields', 'Jobpost_SalaryCanBeShown',
       'Jobpost_MinSalary', 'Jobpost_MaxSalary', 'Jobpost_ProvinceFa',
       'Company_ProvinceFa', 'Company_CityFa',
       'Comany_CompanyOwnershipTypesFa', 'Company_SizeFa',
       'Company_ActivityTypeFa', 'Company_IndustryFa',
       'Company_AgeFromEstablishmentYear', 'Jobpost_ActivationTime_YEAR_MONTH',
       'Jobpost_PriorityWithLocalCandidate'],
      dtype='object')

In [73]:
df_new.isnull().sum()

Jobpost_RawTitle                                   0
Jobpost_MainJobCategory                            0
Jobpost_IndustryFa                                 0
Jobpost_WorkTypeFa                                 0
Jobpost_IsRemote                                   0
Jobpost_IsInternship                               0
Jobpost_PreferredGender                            0
Jobpost_RequiredExperienceYears                    0
Jobpost_RequiredRelatedExperienceInThisIndustry    0
Jobpost_RequiredMaxAge                             0
Jobpost_RequiredMilitaryServiceCard                0
Jobpost_LanguageSkills                             0
Jobpost_SoftwareSkills                             0
Jobpost_AcademicFields                             0
Jobpost_SalaryCanBeShown                           0
Jobpost_MinSalary                                  0
Jobpost_MaxSalary                                  0
Jobpost_ProvinceFa                                 0
Company_ProvinceFa                            